For questions about using the API, see https://quickstats.nass.usda.gov/api

In [1]:
import json
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm

In [7]:
def get_data(years, states='all'):
    """
    Retrieves data from NASS USDA Quick Stats API. 
    
    Feel free to change what is in the config_url to pull different data, 
    and feel free to change the columns that are displayed in the output df.
    Available features are listed in the docs above.
    NOTE: If a requested state has no data for a given year, it is simply skipped.
    
    Args:
        years (list): List of years to retrive. 
        states (list or str): List of states to retrieve, or 'all'
    
    Returns:
        (DataFrame): dataframe concatenated by state then year
    """
    api_key = '986A28EC-EF35-31C1-A179-461CF9DF4F8E'
    root_url = 'http://quickstats.nass.usda.gov/api/api_GET/?key={}'.format(api_key)
    
    if states == 'all':
        states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
                  "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
                  "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
                  "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
                  "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
        
    all_dfs = []
    with tqdm(total=len(years)*len(states)) as pbar:
        for year in years:
            for state in states:
                # Parameters
                source_desc = 'SURVEY' # Program
                sector_desc = 'CROPS' # Sector
                group_desc = 'FIELD CROPS' # Group
                agg_level_desc = 'COUNTY' # Geographical Level
                year = year # Year
                freq_desc = 'ANNUAL' # Period Type
                reference_period_desc = 'YEAR' # Period
                statisticcat_desc = 'AREA HARVESTED' # Data Item
                state_alpha = state

                params = [source_desc, sector_desc, group_desc, agg_level_desc,
                          year, freq_desc, reference_period_desc, statisticcat_desc,
                          state_alpha]

                config_url = '&source_desc={}'\
                             '&sector_desc={}'\
                             '&group_desc={}'\
                             '&agg_level_desc={}'\
                             '&year={}'\
                             '&freq_desc={}'\
                             '&reference_period_desc={}'\
                             '&statisticcat_desc={}'\
                             '&state_alpha={}'.format(*params)

                response = requests.get(root_url + config_url)
                if response.status_code == 200:
                    data = json.loads(response.content.decode('utf-8'))
                    df = pd.DataFrame(data['data'])

                    ###########################################################################
                    # If you want to change the output dataframe, change this list of columns. 
                    # Possible columns can be found at the above URL under "Usage"
                    ###########################################################################
                    df['fips_code'] = df['state_fips_code'] + df['county_code'] # Note: This might not actually be a valid FIPS code in some cases
                    columns = ['year', 'fips_code', 'state_alpha', 
                               'county_name', 'commodity_desc', 'unit_desc', 'Value']
                    df = df[columns]
                    df = df.rename(columns={'Value':'value'})
                    df = df.sort_values(by=['fips_code', 'commodity_desc'])

                    all_dfs.append(df)
                pbar.update()
                
    return pd.concat(all_dfs, ignore_index=True)

df = get_data(years=np.arange(2000, 2021), states=['ND'])
df

100%|██████████| 21/21 [00:38<00:00,  1.85s/it]


,year,fips_code,state_alpha,county_name,commodity_desc,unit_desc,value
0,2000,38001,ND,ADAMS,BARLEY,ACRES,"8,200"
1,2000,38001,ND,ADAMS,CANOLA,ACRES,"2,300"
2,2000,38001,ND,ADAMS,CORN,ACRES,"4,000"
3,2000,38001,ND,ADAMS,CORN,ACRES,"4,900"
4,2000,38001,ND,ADAMS,CORN,ACRES,"4,000"
...,...,...,...,...,...,...,...
14073,2020,38998,ND,OTHER COUNTIES,OATS,ACRES,"34,150"
14074,2020,38998,ND,OTHER COUNTIES,SOYBEANS,ACRES,"44,750"
14075,2020,38998,ND,OTHER COUNTIES,WHEAT,ACRES,"171,100"
14076,2020,38998,ND,OTHER COUNTIES,WHEAT,ACRES,"251,080"
